In [1]:
# MISC
import os
import sys

sys.path.append("..")
os.environ['MUJOCO_GL']='egl'
os.environ['CUDA_VISIBLE_DEVICES']='0'

# import shutup
# shutup.please()

import jax
import jax.numpy as jnp

import numpy as np
from functools import partial

# VIS
import matplotlib.pyplot as plt
import mediapy
import scienceplots
from rich.pretty import pprint
from tqdm.auto import tqdm

from utils.datasets import GCDataset
from agents.dynamics_aware_iql import GCIQLAgent
from hydra import initialize, compose
from omegaconf import OmegaConf

plt.style.use(['seaborn-v0_8-colorblind', 'seaborn-v0_8-notebook'])

GLOBAL_KEY = jax.random.key(42)

In [2]:
from gymnasium.envs.registration import register
from envs.mujoco.online_env_utils import make_online_env
from envs.mujoco.ant import AntEnv

register(
    id='myant',
    entry_point=AntEnv,
    max_episode_steps=256,
    reward_threshold=-3.75,
)

env = make_online_env("myant", default_ind=2)
env.reset()

(array([ 0.8407875 ,  1.0446167 , -0.06372512,  0.0610762 , -0.00720263,
        -0.06552991, -0.07873831,  0.04110082, -0.08881806, -0.04805898,
        -0.0475017 ,  0.09384032, -0.09080444,  0.21484635, -0.0908604 ,
         0.01033072, -0.11167552,  0.01285597, -0.22223855, -0.06412087,
        -0.08989524,  0.08733132, -0.07626024,  0.29255911, -0.06569624,
        -0.01410481, -0.15665607,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0. 

In [31]:
import numpy as np

selected_keys = ["observations", "next_observations", "actions", "done", "reward"]
data = np.load("/home/m_bobrin/ZeroShotRL/envs/mujoco/data_custom_ant/default_ind_0.npz", allow_pickle=True)
data = {key: data[key] for key in selected_keys if key in data.files}

# data1 = np.load("/home/m_bobrin/ZeroShotRL/envs/mujoco/data_custom_ant/default_ind10.npz", allow_pickle=True)
# data1 = {key: data1[key] for key in selected_keys if key in data1.files}

In [27]:
import jax
import jax.numpy as jnp
test_d = jax.tree.map(lambda x, y: jnp.concatenate([x, y], axis=0), data, data1)

In [28]:
jax.tree.map(lambda x: x.shape, test_d)

{'action': (20, 256, 8), 'done': (20, 256), 'reward': (20, 256)}